In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = 'data/facies_vectors.csv'
training_data_raw = pd.read_csv(filename)
training_data_raw

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
4144,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
4145,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
4146,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
4147,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


## Loading and adding the variables from the dwt

In [2]:
new_vars = pd.read_pickle('data/vars_from_dwt.pkl')
new_vars

,Depth,GR_cD_step_level_4,Well Name
0,2793.0,0.062591,SHRIMPLIN
1,2793.5,-1.930541,SHRIMPLIN
2,2794.0,-1.930541,SHRIMPLIN
3,2794.5,-5.282836,SHRIMPLIN
4,2795.0,-5.282836,SHRIMPLIN
...,...,...,...
4144,3120.5,1.743657,CHURCHMAN BIBLE
4145,3121.0,-0.891801,CHURCHMAN BIBLE
4146,3121.5,-0.891801,CHURCHMAN BIBLE
4147,3122.0,0.956644,CHURCHMAN BIBLE


In [3]:
new_vars2 = pd.read_pickle('data/vars_from_log_entropy.pkl')
new_vars2

,Depth,ILD_log10_entropy_foot20,DeltaPHI_entropy_foot20,PHIND_entropy_foot20,Well Name
0,2793.0,1.000000,1.000000,1.000000,SHRIMPLIN
1,2793.5,1.584963,1.584963,1.584963,SHRIMPLIN
2,2794.0,1.584963,1.584963,1.584963,SHRIMPLIN
3,2794.5,1.584963,1.584963,1.584963,SHRIMPLIN
4,2795.0,1.584963,1.584963,1.584963,SHRIMPLIN
...,...,...,...,...,...
4144,3120.5,1.584963,0.918296,0.000000,CHURCHMAN BIBLE
4145,3121.0,1.584963,1.584963,0.000000,CHURCHMAN BIBLE
4146,3121.5,1.584963,1.584963,0.000000,CHURCHMAN BIBLE
4147,3122.0,1.584963,1.584963,0.000000,CHURCHMAN BIBLE


In [4]:
# new_vars3 = pd.read_pickle('data/vars_from_gradient.pkl')
# new_vars3

In [5]:
new_vars = new_vars.drop(['Well Name','Depth'],axis=1)
new_vars2 = new_vars2.drop(['Well Name','Depth'],axis=1)
# new_vars3 = new_vars3.drop(['Well Name','Depth'],axis=1)

training_data = pd.concat([training_data_raw,new_vars,new_vars2],axis=1)
training_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,GR_cD_step_level_4,ILD_log10_entropy_foot20,DeltaPHI_entropy_foot20,PHIND_entropy_foot20
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000,0.062591,1.000000,1.000000,1.000000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979,-1.930541,1.584963,1.584963,1.584963
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957,-1.930541,1.584963,1.584963,1.584963
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936,-5.282836,1.584963,1.584963,1.584963
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915,-5.282836,1.584963,1.584963,1.584963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685,1.743657,1.584963,0.918296,0.000000
4145,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677,-0.891801,1.584963,1.584963,0.000000
4146,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669,-0.891801,1.584963,1.584963,0.000000
4147,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661,0.956644,1.584963,1.584963,0.000000


In [6]:
blind = training_data[training_data['Well Name'] == 'SHANKLE']
training_data = training_data[(training_data['Well Name'] != 'SHANKLE')]

The rows of the confusion matrix correspond to the actual facies labels.  The columns correspond to the labels assigned by the classifier.  For example, consider the first row. For the feature vectors in the test set that actually have label `SS`, 23 were correctly indentified as `SS`, 21 were classified as `CSiS` and 2 were classified as `FSiS`.

The entries along the diagonal are the facies that have been correctly classified.  Below we define two functions that will give an overall value for how the algorithm is performing.  The accuracy is defined as the number of correct classifications divided by the total number of classifications.

In [7]:
def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
    acc = total_correct/sum(sum(conf))
    return acc

As noted above, the boundaries between the facies classes are not all sharp, and some of them blend into one another.  The error within these 'adjacent facies' can also be calculated.  We define an array to represent the facies adjacent to each other.  For facies label `i`, `adjacent_facies[i]` is an array of the adjacent facies labels.

In [8]:
adjacent_facies = np.array([[1], [0,2], [1], [4], [3,5], [4,6,7], [5,7], [5,6,8], [6,7]])

def accuracy_adjacent(conf, adjacent_facies):
    nb_classes = conf.shape[0]
    total_correct = 0.
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
        for j in adjacent_facies[i]:
            total_correct += conf[i][j]
    return total_correct / sum(sum(conf))

## Applying the classification model to the blind data

We held a well back from the training, and stored it in a dataframe called `blind`:

In [9]:
blind

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,GR_cD_step_level_4,ILD_log10_entropy_foot20,DeltaPHI_entropy_foot20,PHIND_entropy_foot20
937,2,A1 SH,SHANKLE,2774.5,98.36,0.642,-0.1,18.685,2.9,1,1.000,-0.064475,1.000000,1.000000,1.000000
938,2,A1 SH,SHANKLE,2775.0,97.57,0.631,7.9,16.745,3.2,1,0.984,-0.064475,1.584963,1.584963,1.584963
939,2,A1 SH,SHANKLE,2775.5,98.41,0.615,12.8,14.105,3.2,1,0.968,4.514241,0.918296,1.584963,1.584963
940,2,A1 SH,SHANKLE,2776.0,85.92,0.597,13.0,13.385,3.4,1,0.952,4.514241,0.000000,1.584963,0.918296
941,2,A1 SH,SHANKLE,2776.5,83.16,0.592,12.3,13.345,3.4,1,0.935,-1.361929,0.000000,1.584963,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,8,C LM,SHANKLE,3006.0,32.84,1.120,-2.2,3.455,5.1,2,0.060,18.846750,1.584963,0.000000,0.000000
1382,8,C LM,SHANKLE,3006.5,25.16,1.112,-1.6,2.890,4.8,2,0.045,-22.968646,0.918296,0.000000,0.000000
1383,8,C LM,SHANKLE,3007.0,25.16,1.112,-1.6,2.890,4.8,2,0.030,-22.968646,0.918296,0.000000,0.000000
1384,4,C LM,SHANKLE,3007.5,26.22,1.092,-0.4,3.400,4.5,2,0.030,2.152815,1.584963,0.000000,0.000000


The label vector is just the `Facies` column:

In [10]:
y_blind = blind['Facies'].values -1
y_blind

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 7, 7, 7, 7, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 5, 5, 5, 5,
       5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       1, 1, 1, 1, 1, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 2, 2, 2,
       2, 2,

We can form the feature matrix by dropping some of the columns and making a new dataframe:

In [11]:
well_features = blind.drop(['Facies', 'Formation', 'Well Name', 'Depth'], axis=1)

In [12]:
X_blind = well_features
X_blind.fillna(value=-9999,inplace=True)
X_blind = X_blind.values

In [13]:
training_data.fillna(value=-9999,inplace=True)
len(training_data)

3700

In [14]:
X_train = training_data.drop(['Facies', 'Formation', 'Well Name', 'Depth'], axis=1).values
y_train = training_data['Facies'].values-1
y_train

array([2, 2, 2, ..., 4, 4, 4])

In [15]:
from imblearn.over_sampling import SMOTE

In [16]:
smt = SMOTE()
X_train, y_train = smt.fit_sample(X_train,y_train)

Now it's a simple matter of making a prediction and storing it back in the dataframe:

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
from sklearn import metrics
y_pred = clf.predict(X_blind)
blind['Prediction'] = y_pred
score = metrics.f1_score(y_blind, y_pred, average='weighted')
print(score)

0.495331146493


Let's see how we did with the confusion matrix:

In [20]:
from sklearn.metrics import confusion_matrix
from classification_utilities import display_cm, display_adj_cm

In [21]:
cv_conf = confusion_matrix(y_blind, y_pred)

print('Optimized facies classification accuracy = %.3f' % accuracy(cv_conf))
# print('Optimized adjacent facies classification accuracy = %.3f' % accuracy_adjacent(cv_conf, adjacent_facies))

Optimized facies classification accuracy = 0.521
